# Loading the `Moral-Stories` dataset
***
The dataset and code can be found <a href="https://github.com/demelin/moral_stories">here</a>.\
The authors provide 12k unique norms and, for some reason, additional 700k variations of the same norms, just with NaN fields every now and then. Zero additional information, but maybe I am overlooking something here?
* Might be for different tasks? But then they only provide a single label which is always 1 for any NaN rows...

# Sample task: Action classification
***
For starters, let's reproduce a task from the paper:
* Given an action, predict whether it is moral or immoral.
* For simplicity, we do not use the splits introduced in the paper, but rather random splitting

We start by loading the data as a `pandas.DataFrame`:

In [12]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
from ailignment.datasets import get_accuracy_metric, join_sentences, tokenize_and_split
import pandas as pd
import datasets
import transformers
import numpy as np
from ailignment import sequence_classification

#transformers.logging.set_verbosity_warning()

from collections import Counter

import spacy
from spacy import displacy
from tqdm import tqdm
import pandas as pd

In [ ]:
dataframe = get_moral_stories()
nlp = spacy.load("en_core_web_sm")
# run everything through spacy for part of speech tags
dataframe["norm_parsed"] = dataframe["norm"].apply(nlp)

In [ ]:
def get_pos_until_verb(doc):
    '''
    Returns a tuple of Part-of-speech tags up until the first verb
    Returns ("EMPTY") if no VERB is present
    '''
    l = [d.pos_ for d in doc]
    if "VERB" not in l: return ("EMPTY",)
    return tuple(l[:l.index("VERB")])

In [ ]:
dataframe["norm_pos"] = dataframe["norm_parsed"].apply(lambda x: [d.pos_ for d in x])

In [ ]:
dataframe["pos_verb"] = dataframe["norm_parsed"].apply(get_pos_until_verb)
#pos_verb = dataframe.groupby("pos_verb")

# Untying the *is* from the *ought*
***
We are not to come up with new norms. Rather, we like to get rid of the usually ambiguous normative judgements. Hence, we aim to split the moral value assigned to a norm from the situation it applies to. Checking for norm violation then reduces to other well-known problems such as question answering or textual entailment.

**TODO:** Surely there are smart names for the two categories of *situations that are judge-worthy* and their corresponding moral values.

We look for the most common ways that norms are phrased:
* "You should" or "You shouldn't"
* "It's" or "It is" followed by good, bad, etc.

Then, we want to translate them into "Did you ....?" and "Well, that's ..." pieces
* E.g. "You should back up so you can let people park." becomes
    * "Did you backup so you can let people park?"
    * "Yes" -> "Good"
    * "No" -> "Well, you should have!"

The norms only give value for either violation or adherence, but rarely both. We mirror the sentiment of the judgement by simple negation:
* "It is bad to hurt people"
    * "Did you hurt people?"
    * "Yes" --> "bad"
    * "No" --> "not bad"

Note, that we refrain from translating "not bad" to "good", since norms often aren't morally symmetric, i.e. not every norm has equally strong normativity for violation and adherence. E.g. positively committing a crime is punished much more explicitly than is not committing it. 

In [ ]:
norms = dataframe["norm"].apply(str.lower)

f_should = lambda x: x.startswith("you should ") and not x.startswith("you should not")
shoulds = dataframe[norms.apply(f_should)]
print("\"You should\":", len(shoulds))

f_shouldnt = lambda x: x.startswith("you shouldn") or x.startswith("you should not")
shouldnts = dataframe[norms.apply(f_shouldnt)]
print("\"You shouldnt\":", len(shouldnts))

f_its = lambda x: x.startswith("it's") or x.startswith("it is")
its = dataframe[norms.apply(f_its)]
print("\"It's\"", len(its))

total = len(its)+len(shoulds)+len(shouldnts)
print("Covered", total ,"of",len(dataframe), f"({100*total/len(dataframe):.2f}%) norms")

### Untying `you should` sentences
***
First, find the most common grammatical structures:

In [ ]:
common_pos = shoulds["norm_pos"].apply(lambda x: tuple(x[:3]))
common_pos = shoulds.groupby(common_pos)
most_common = common_pos["ID"].count().sort_values(ascending=False)
most_common

Only two seem to be of relevance:
* "You should VERB"
    * You should help the ones in need
* "You should ADV"
    * You should always, You should never

While the first is straightforward to de-value, the latter at least requires more sophisticated reasoning capabilities concerning quantification and negation of cases.

For now, we simply strip the initial "You should" from the former, and the "You should ADV" from the latter to obtain the judgements.

In [ ]:
# do first group of (PRON, AUX, VERB)
g0 = common_pos.get_group(("PRON","AUX","VERB")).copy()
g0["norm_devalued"] = g0["norm_parsed"].apply(lambda x: x[2:].text)
g0["value"] = "You should"

# second group of (PRON, AUX, ADV)
g1 = common_pos.get_group(("PRON","AUX","ADV")).copy()
g1["norm_devalued"] = g1["norm_parsed"].apply(lambda x: x[3:].text)
g1["value"] = g1["norm_parsed"].apply(lambda x: "You should " + x[2].text)

shoulds_devalued = pd.concat([g0,g1])

### Untying `you should not` sentences
***
First, find the most common grammatical structures:

In [ ]:
common_pos = shouldnts["norm_pos"].apply(lambda x: tuple(x[:4]))
common_pos = shouldnts.groupby(common_pos)
most_common = common_pos["ID"].count().sort_values(ascending=False)
most_common

Only one seems to be of relevance:
* "You should not VERB"
    * You should not spit in someone's face

Similarly to the "You should" cases, we strip "You should not" as the value-judgement.

In [ ]:
shouldnts_devalued = common_pos.get_group(("PRON","AUX","PART", "VERB")).copy()
shouldnts_devalued["norm_devalued"] = shouldnts_devalued["norm_parsed"].apply(lambda x: x[3:].text)
shouldnts_devalued["value"] = "You should not"

### Untying `it is` sentences
***
First, find the most common grammatical structures up until the first verb:

In [ ]:
#common_pos = its["norm_pos"].apply(lambda x: tuple(x[:5]))
common_pos = its.groupby("pos_verb")
most_common = common_pos["ID"].count().sort_values(ascending=False)
most_common[:20]

The situation is a little more complex here, it seems:
1. "It is ADJ to", It's wrong to become addicted to gambling.
    * Value: ADJ, strip until first verb
2. "It is not ADJ to", It's not okay to invade someone else's privacy.
    * Like 1.
3. (PRON, AUX, ADJ, PART, PART). Two subgroups, which are both currently ignored
    1. "It is ADJ not to"
    2. "It is ADJ to not"

Similarly to the "You should" cases, we strip "You should not" as the value-judgement.

In [ ]:
groups = []
# (PRON, AUX, ADJ, PART)
g = common_pos.get_group(("PRON","AUX","ADJ", "PART")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[4:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2].text)
groups.append(g)

In [ ]:
# (PRON, AUX, PART, ADJ, PART)
g = common_pos.get_group(("PRON","AUX","PART","ADJ", "PART")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[5:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2:4].text)
groups.append(g)

In [ ]:
# (PRON, AUX, ADJ, PART, PART)
g = common_pos.get_group(("PRON","AUX","ADJ","PART", "PART")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[5:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2:4].text)
# currently ignored

In [ ]:
# (PRON, AUX)
g = common_pos.get_group(("PRON","AUX")).copy()
g["norm_devalued"] = g["norm_parsed"].apply(lambda x: x[5:].text)
g["value"] = g["norm_parsed"].apply(lambda x: x[2:4].text)
# currently ignored

In [ ]:
# finally, stitch everything together
its_devalued = pd.concat(groups)

# Imperatives and values
***
By splitting up value-judgements from the norms we end up with imperatives describing what the action looks like and the value one should expect to receive from it.

In [ ]:
dataframe_devalued = pd.concat([shoulds_devalued, shouldnts_devalued, its_devalued])
total_split = len(dataframe_devalued)
print("After devaluation, we cover", total_split ,"of",len(dataframe), f"({100*total_split/len(dataframe):.2f}%) norms")

In [ ]:
dataframe_devalued[["moral_action","norm_devalued","value"]].sample(10)

# Vertical protoype
***
Before bothering with the norms that were left out during the above steps, I'd like to show a proof of concept. Therefore, the next steps are:
1. Derive a set of subjectified actions from the normative imperatives. I will do this with a table of conjugated verbs and handcrafted rules for the pronouns you, your and yourself. What about the names??
2. Run the textual entailment experiment between pairs of subjectified action and either moral or immoral action

## Subjectified actions prototype
***
1. Get a list of conjugated verbs
2. Get all norms whose imperative verb is contained in the list
3. Find the actor's name from the actions


In [ ]:
# load a list of conjugated english verbs from https://github.com/monolithpl/verb.forms.dictionary
verbs = pd.read_csv("https://raw.githubusercontent.com/monolithpl/verb.forms.dictionary/master/csv/verbs-dictionaries.csv",
                   sep="\t", 
                    names=["present simple 1st","present simple 3rd","past simple","past participle","present participle"])
base_forms = set(verbs["present simple 1st"].to_list())
verb_map = {a:b for i,(a,b) in verbs[verbs.columns[:2]].iterrows()}

In [ ]:
# re run pos tagging on the imperatives
dataframe_devalued["norm_devalued_pos"] = dataframe_devalued["norm_devalued"].apply(nlp)

In [ ]:
# filter out all norms for which we do not have an entry in our verb list
norm_verbs = dataframe_devalued["norm_devalued_pos"].apply(lambda x: x[0].text)
dataframe_devalued = dataframe_devalued[norm_verbs.apply(lambda x: x in base_forms)]
print("After filtering out unknown verbs, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

In [ ]:
# apply pos tagging to moral actions to find sentences starting with a name
dataframe_devalued["moral_action_parsed"] = dataframe_devalued["moral_action"].apply(nlp)

In [ ]:
# find the most common pos tags at the start of the actions
pos_groups = dataframe_devalued.groupby(dataframe_devalued["moral_action_parsed"].apply(lambda x: x[0].pos_))
counts = pos_groups["ID"].count().sort_values(ascending=False)
print(counts)

In [ ]:
# after visual examination, the first three categories seem to only contain names
# pos_groups["moral_action"].get_group(counts.index[2]).to_list()
dataframe_devalued = pd.concat([pos_groups.get_group(counts.index[i]) for i in [0,1,2]])
print("After focusing on known names, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

In [ ]:
# next up, gather actor names
dataframe_devalued["actor_name"] = dataframe_devalued["moral_action_parsed"].apply(lambda x: x[0].text)

### Switching persons
***
We need to also translate the pronouns in the imperatives from second (you, your) to third person (his/hers, him/her). Unfortunately, this requires the gender of the actor.

Here, we use package `gender-guesser`, as found here https://github.com/lead-ratings/gender-guesser

It assigns either andy, female, male, mostly female/male or unknown gender.

Once again, the ambiguity of the english language hits us:
* "you" might either stand for he/she or him/her
* Since only around 400 rows are affected, we ignore them for the protoype

In [ ]:
import gender_guesser.detector as gg
det = gg.Detector()
dataframe_devalued["actor_gender"] = dataframe_devalued["actor_name"].apply(lambda x: det.get_gender(x,"usa"))
print(dataframe_devalued["actor_gender"].groupby(dataframe_devalued["actor_gender"]).count().sort_values(ascending=False))

In [ ]:
# filter on genders
dataframe_devalued["actor_gender"].replace("mostly_male","male", inplace=True)
dataframe_devalued["actor_gender"].replace("mostly_female","female", inplace=True)

dataframe_devalued = dataframe_devalued[dataframe_devalued["actor_gender"].apply(lambda x: "male" in x)]
print("After focusing on known genders, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

In [ ]:
# get rid of all sentences with a "you", since we can't properly conjugate(?) it
dataframe_devalued = dataframe_devalued[dataframe_devalued["norm_devalued"].apply(lambda x: " you " not in x and not x.endswith(" you"))]
print("After filtering 'you's from norms, we cover", len(dataframe_devalued) ,"of",len(dataframe), 
      f"({100*len(dataframe_devalued)/len(dataframe):.2f}%) norms")

In [ ]:
# we check whether a norm story contains any 2nd person pronouns
male_pron_map = {"you":"he","your":"his","yours":"his","yourself":"himself"}
female_pron_map = {"you":"she","your":"her","yours":"hers","yourself":"herself"}

def translate_row(row):
    # actor name + 3rd person verb + rest of imperative
    # replace pronouns with 3rd person version
    verb = verb_map[row["norm_devalued_pos"][0].text]
    
    doc = row["norm_devalued_pos"]
    pron_map = male_pron_map if row["actor_gender"] == "male" else female_pron_map
    imperative = doc[:len(doc) if not doc[-1].is_punct else len(doc)-1]
    imperative = " ".join([pron_map.get(x.text, x.text) for x in imperative[1:]])

    story = " ".join([row["actor_name"], verb, imperative])
    return story


dataframe_devalued["norm_story"] = dataframe_devalued.apply(translate_row, axis=1)

# Testing the prototype
***
So far, we obtained several situations of people actually performing the norm, which we call `norm-story`. Now, we'd like to test, whether natural language inference (NLI), or more specifically, textual entailment models are able to tell apart the moral action from the immoral ones:
* Norm: *It's manipulative to try to force a partner into marriage.*
* Norm-story: *Jake tries to force a partner into marriage*
* Moral action: *Jake proposes to Harry at the bar they met at.*
* Immoral action: *Jake tells Harry that he will kill himself if he doesn't marry him.*

General idea: If an action $A$ entails the norm-story $A_N$ we assume that the actor also performed $A_N$ and is therefore exposed to the value-judgement of the norm. Continuing the above example:
* If we find that $A=$*Jake proposes to Harry at the bar they met at.* is a sufficient condition for the statement $A_N=$*Jake proposes to Harry at the bar they met at.*, then we assign the value *manipulative* to $A$.

In order to decide whether $A$ is moral or immoral, the worded value (e.g. manipulative, good, bad) needs to be interpretet as moral or immoral. Here we make the case, that sentiment analysis expresses whether a word was meant to be positive or negative. Consequently, we use the estimated sentiments as numerical normative values.

In [ ]:
# Estimate sentiments using huggingface
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
value_map = {v:classifier(v)[0]["label"] for v in dataframe_devalued["value"].unique()}
dataframe_devalued["norm_sentiment"] = dataframe_devalued["value"].apply(value_map.get)

In [ ]:
# save a stripped version of the data for later use
dataframe_devalued.to_pickle("../data/moral_stories_proto.dat")

In [2]:
dataframe_devalued = pd.read_pickle("../data/moral_stories_proto.dat")

In [3]:
# load the NLI model and its tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli" # 85%
#name = 'cross-encoder/nli-distilroberta-base' # 80%
#name = "boychaboy/SNLI_bert-base-uncased" # 75%
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSequenceClassification.from_pretrained(name)

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# create a huggingface dataset and setup metrics
from datasets import Dataset

def tok(samples):
    return tokenizer(samples["moral_action"], samples["norm_story"], padding="max_length", 
                     truncation=True, return_token_type_ids=True)

data = dataframe_devalued[["moral_action", "norm_story"]].copy()
data["label"] = dataframe_devalued["norm_sentiment"].apply(lambda x: int(x=="POSITIVE"))
dataset = Dataset.from_pandas(data)
dataset = dataset.map(tok, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

In [5]:
eval_set = dataset

In [6]:
# run evaluation
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=0,              # total number of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,                # how often to log
    evaluation_strategy="epoch",     # when to run evaluation
)

trainer = Trainer(
    model=model,
    args=training_args,
)
results = trainer.predict(eval_set)
scores = torch.softmax(torch.from_numpy(results.predictions),1).numpy()

is_entailed = (scores[:,0] > scores[:,2]).astype("int32")
labels = np.array(eval_set["label"])
acc = (is_entailed == labels).sum()/len(labels)
print("Accuracy:", acc)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, moral_action, norm_story.
***** Running Prediction *****
  Num examples = 8445
  Batch size = 16


Accuracy: 0.8525754884547069


In [13]:
is_entailed = (scores[:,2] > scores[:,1]).astype("int32")
labels = np.array(eval_set["label"])
acc = (is_entailed == labels).sum()/len(labels)
print("Accuracy:", acc)

Accuracy: 0.4389579632918887


In [14]:
from transformers import Trainer, TrainingArguments
import torch

dataframe = dataframe_devalued[dataframe_devalued.columns[:8]]
test_split = 0.2
batch_size = 12
model = "distilbert-base-uncased"
#model = "albert-base-v2"
action_dataframe = make_action_classification_dataframe(dataframe)
input_columns = ["norm", "action"]
action_dataframe["task_input"] = join_sentences(action_dataframe, input_columns)
dataset = datasets.Dataset.from_pandas(action_dataframe)
dataset = dataset.train_test_split(test_size=test_split)

def data_all(tokenizer):
    return tokenize_and_split(dataset, tokenizer, "task_input")
def data_small(tokenizer):
    train, test = data_all(tokenizer)
    train = train.shuffle(seed=42).select(range(1000))
    test = test.shuffle(seed=42).select(range(1000))
    return train, test

training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    #weight_decay=wd,
    #learning_rate=lr,
    logging_dir='./logs',
    logging_steps=500,
    save_steps=1000000,
    save_total_limit=0,
    evaluation_strategy="epoch",
)

r = sequence_classification(data_all, model, training_args, get_accuracy_metric())
acc = [x["eval_accuracy"] for x in r if "eval_accuracy" in x]
print(acc)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\nikla/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: intention, norm, action, situation, ID, task_input, consequence.
***** Running training *****
  Num examples = 13512
  Num Epochs = 3
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 3378


Epoch,Training Loss,Validation Loss,Accuracy
1,0.522400,0.489160,0.780935
2,0.345300,0.471897,0.790705
3,0.193300,0.762193,0.799882


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: intention, norm, action, situation, ID, task_input, consequence.
***** Running Evaluation *****
  Num examples = 3378
  Batch size = 12
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: intention, norm, action, situation, ID, task_input, consequence.
***** Running Evaluation *****
  Num examples = 3378
  Batch size = 12
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: intention, norm, action, situation, ID, task_input, consequence.
***** Running Evaluation *****
  Num examples = 3378
  Batch size = 12


Training completed. Do not forget to share your model on huggingface.co/models =)




[0.7809354647720544, 0.790704558910598, 0.7998815867377146]


In [ ]:
[[x["eval_accuracy"] for x in r if "eval_accuracy" in x] for r in results]

# WIP: Get score output from LM
***
Question: Is there a better way to sample from generated LM outputs?

In [ ]:
from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelWithLMHead.from_pretrained(model)

prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=False, top_p=0.95, top_k=60,
                        return_dict_in_generate=True, output_attentions=False,
                        output_hidden_states=True, output_scores=True)
#generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

p = torch.softmax(outputs.scores[0], dim=1)

print(p.max())

# WIP: Data augmentation with NER
***
Idea: Use Named entity recognition to find and replace persons etc. 

In [ ]:
from ailignment.datasets.moral_stories import get_moral_stories, make_action_classification_dataframe
from ailignment import join_sentences, tokenize_and_split, get_accuracy_metric
dataframe = get_moral_stories()
columns = dataframe.columns[1:]
print("Running NER on columns", columns.to_list())

In [ ]:
texts = join_sentences(dataframe ,columns, "\n")

In [ ]:
import spacy
from spacy import displacy
from tqdm import tqdm
import pandas as pd

nlp = spacy.load("en_core_web_sm")

ner_pipe = nlp.pipe(tqdm(texts), disable=['tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
docs = [x for x in ner_pipe]

displacy.render(docs[0], style="ent")

In [ ]:
from collections import Counter

def get_frequent_entity(doc, entity="PERSON", n=1):
    '''
    Returns the highest number of occurences of an
    entity in the NER doc.
    '''
    occurences = [(x.text, x.label_) for x in doc.ents if x.label_ == entity]
    c = Counter(occurences)
    ents = []
    for item, count in c.most_common(n):
        ents.append([x for x in doc.ents if (x.text, x.label_) == item])
    
    if n == 1 and len(ents) != 0:
        ents = ents[0]
    return ents

In [ ]:
persons = [get_frequent_entity(x, "PERSON",n=1) for x in docs]
# we are interested in the simplest case, where the NER found
# exactly 6 matches
matches = [x for x in persons if len(x) == 6]
print(f"Found {len(matches)} matches")

In [ ]:
m = matches[0]
displacy.render(m[0].doc, "ent")

In [ ]:
def replace_entity(ents, s):
    '''
    Replaces all occurences of entities in `ents` with `s`.
    `ents` is a list of entities as returned by `doc.ents`
    from an NER pipeline, they need to be from the same doc!
    '''
    offset = 0
    text = ents[0].doc.text
    new_text = ""
    for ent in ents:
        start = ent.start_char
        end = ent.end_char
        left = text[offset:start]
        new_text += left + s
        offset = end
    new_text += text[offset:]
    return new_text

In [ ]:
n_docs = [replace_entity(m, "Niklas").split("\n") for m in matches]

In [ ]:
n_docs = [m[0].doc.text.split("\n") for m in matches]

In [ ]:
dataframe_replaced = pd.DataFrame(n_docs)
dataframe_replaced.columns = columns
dataframe_replaced.head()

In [ ]:
import datasets
test_split = 0.2
batch_size = 8

action_dataframe = make_action_classification_dataframe(dataframe_replaced)

input_columns = ["action"]
action_dataframe["task_input"] = join_sentences(action_dataframe, input_columns, " ")
dataset = datasets.Dataset.from_pandas(action_dataframe)
dataset = dataset.train_test_split(test_size=test_split)


In [ ]:

from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model)

train_data, test_data = tokenize_and_split(dataset, tokenizer, "task_input")

# for prototyping, optional
small_train_data = train_data.shuffle(seed=42).select(range(1000))
small_test_data = test_data.shuffle(seed=42).select(range(1000))

training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,                # how often to log
    save_steps=1000,
    save_total_limit=0,
    evaluation_strategy="epoch",     # when to run evaluation
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=small_train_data,   # training dataset
    eval_dataset=small_test_data,     # evaluation dataset
    compute_metrics=get_accuracy_metric,     # code to run accuracy metric
)
trainer.train()

In [ ]:
from gender_guesser.detector import Detector

In [ ]:
d = Detector()

In [ ]:
d.get_gender("Jamie")

In [ ]:
from ailignment.datasets.moral_stories import _lemmatize, get_moral_stories

In [ ]:
import spacy
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'textcat'])
STOP_WORDS = stopwords.words('english')
stories = get_moral_stories()
columns = ["moral_action", "immoral_action"]


In [ ]:
series = stories[columns[0]]

In [ ]:
def lemmatize_series(series, nlp, STOP_WORDS=None):
    '''
    Given a series of strings, returns a DataFrame(["lemmas", "tokens", "maps"])
    of the lemmatized strings according to `_lemmatize` function.
    '''
    # get rid of whitespace
    translation_table = str.maketrans(' ', ' ', punctuation)
    series = series.map(lambda x: x.translate(translation_table))
    series = series.map(lambda x: _lemmatize(x, nlp, STOP_WORDS))
    data = pd.DataFrame(series.to_list(), columns=["lemmas", "tokens", "maps"])
    return data

In [ ]:
y = series.map(lambda x: _lemmatize(x, nlp, STOP_WORDS))

In [ ]:
data = pd.DataFrame(y.to_list(), columns=["lemmas", "tokens", "maps"])


In [ ]:
data

In [ ]:
b

In [ ]:
c